In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

C:\Users\kajte\anaconda3\envs\py39_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.10.0
0.11.1


'NVIDIA GeForce GTX 1650'